In [99]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics, svm
from IPython.display import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV
from nltk.stem import SnowballStemmer
import string
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline  

## Dataset
Pandas provide a .read_csv() function. the paramter engine='python' was needed here.
We can see in the output that there are two columns "text" and "spam" 

In [100]:
data = pd.read_csv('/home/shreyansh/Downloads/emails.csv', encoding='latin-1', engine='python')
data.head(n=10)
print(data)

                                                   text  spam
0     Subject: naturally irresistible your corporate...     1
1     Subject: the stock trading gunslinger  fanny i...     1
2     Subject: unbelievable new homes made easy  im ...     1
3     Subject: 4 color printing special  request add...     1
4     Subject: do not have money , get software cds ...     1
5     Subject: great nnews  hello , welcome to medzo...     1
6     Subject: here ' s a hot play in motion  homela...     1
7     Subject: save your money buy getting this thin...     1
8     Subject: undeliverable : home based business f...     1
9     Subject: save your money buy getting this thin...     1
10    Subject: las vegas high rise boom  las vegas i...     1
11    Subject: save your money buy getting this thin...     1
12    Subject: brighten those teeth  get your  teeth...     1
13    Subject: wall street phenomenon reaps rewards ...     1
14    Subject: fpa notice : ebay misrepresentation o...     1
15    Su

In [101]:
def stemmer (text):
    text = text.split()
    words = ""
    for i in text:
            stemmer = SnowballStemmer("english")
            words += (stemmer.stem(i))+" "
    return words

In [102]:
data['text']=data['text'].apply(stemmer)

In [103]:
f = TfidfVectorizer(stop_words = 'english')
X = f.fit_transform(data["text"])
print(np.shape(X))
print(X)

(5728, 29101)
  (0, 25144)	0.018320780122310316
  (0, 18486)	0.0774666020541566
  (0, 14727)	0.14736976038474162
  (0, 8307)	0.07050131715310318
  (0, 14000)	0.0938151598024009
  (0, 16675)	0.11580733234546622
  (0, 22001)	0.07011905153908338
  (0, 13168)	0.08063017188146154
  (0, 22083)	0.11454332911694699
  (0, 7890)	0.1626189570288924
  (0, 17093)	0.2048836446936474
  (0, 25314)	0.12724232620438508
  (0, 14361)	0.043518406160338516
  (0, 14766)	0.13875892723664263
  (0, 12687)	0.10557755235592851
  (0, 6943)	0.11788360227052634
  (0, 16521)	0.35477178806462895
  (0, 25125)	0.12724232620438508
  (0, 24867)	0.12605993014920455
  (0, 19665)	0.0938151598024009
  (0, 28031)	0.13683433125504949
  (0, 16922)	0.09319476031976016
  (0, 25653)	0.09482761518737638
  (0, 10162)	0.09546308166869215
  (0, 21334)	0.1759258777737806
  :	:
  (5727, 21686)	0.04263369319795056
  (5727, 14432)	0.11220093044304848
  (5727, 26243)	0.09695542166571573
  (5727, 24064)	0.03723293891927914
  (5727, 6840)	0.0

In [104]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, data['spam'], test_size=0.30, random_state=42)


In [105]:
list_C = np.arange(800,900,100)
list_gamma = np.arange(0.001,0.006,0.001)
parameters = [{'gamma': list_gamma,
                     'C': list_C}]
print(list_C)
print(list_gamma)
print(parameters)

[800]
[0.001 0.002 0.003 0.004 0.005]
[{'gamma': array([0.001, 0.002, 0.003, 0.004, 0.005]), 'C': array([800])}]


In [106]:
svc = svm.SVC(kernel='rbf')
gridsearch = GridSearchCV(svc, parameters, cv=2, scoring='f1', n_jobs=-1)
gridsearch.fit(X_train,y_train)
gridsearch.best_params_

{'C': 800, 'gamma': 0.001}

In [107]:
gridsearch.best_score_

0.9803859402162253

In [108]:
m_confusion_test = metrics.confusion_matrix(y_test, gridsearch.best_estimator_.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

,Predicted 0,Predicted 1
Actual 0,1274,4
Actual 1,14,427
